In [2]:
import os
import sys
import pandas as pd
import json
from dotenv import load_dotenv

# Adiciona a pasta raiz do projeto ao sys.path
# O '..' representa o diretório pai (no caso, a raiz do projeto 'meu_projeto_extracao')
# O os.path.abspath converte para um caminho absoluto
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

# Classe para extração de dados via API (desenvolvida em src/extractors/api_extractor.py)
from src.extractors.api_extractor import ExtratorAPI

In [3]:
# Carrega as variáveis do arquivo .env para o ambiente
load_dotenv()

False

In [4]:
# URL do api
URL_API = "https://api.obrasgov.gestao.gov.br/obrasgov/api/projeto-investimento"

In [5]:
# Parametros iniciais
PARAMS = {
    'uf': 'DF',
    'pagina': 0,          
    'tamanhoDaPagina': 100,     
    'ordenacao': 'ID_UNICO'
}

In [ ]:
extrator = ExtratorAPI(
    url_base=URL_API,
    params=PARAMS,
)

In [ ]:
try:
    # O método 'executar_extracao' cuida de todo o processo
    data, metadata = extrator.executar_extracao()
    
    # --- 3. Salvando os Resultados em JSON ---
    if data: # Apenas salva se algum dado foi coletado
        
        # Define o caminho de saída com a extensão .json
        out_dir = '../data/raw/dados_obras_gov.json'
        
        # Garante que o diretório de saída exista
        os.makedirs(os.path.dirname(out_dir), exist_ok=True)
        
        # Abre o arquivo em modo de escrita ('w') com codificação UTF-8
        with open(out_dir, 'w', encoding='utf-8') as f:
            # Salva a lista de dados no arquivo JSON
            # indent=4 -> Formata o arquivo para ser legível por humanos
            # ensure_ascii=False -> Garante que caracteres especiais (acentos) sejam salvos corretamente
            json.dump(data, f, ensure_ascii=False, indent=4)
        
        print(f"\nDados salvos com sucesso em formato JSON: {out_dir}")
        print(f"Total de registros salvos: {len(data)}")
    else:
        print("\nNenhum dado foi coletado para salvar.")

except Exception as e:
    print(f"\nO processo de extração falhou: {e}")

2025-10-16 14:01:24 - INFO - Iniciando extração da API em 'https://api.obrasgov.gestao.gov.br/obrasgov/api/projeto-investimento'


Páginas extraídas: 0 pág [00:00, ? pág/s]

2025-10-16 14:01:25 - INFO - Requisitando URL: https://api.obrasgov.gestao.gov.br/obrasgov/api/projeto-investimento?uf=DF&pagina=0&tamanhoDaPagina=100&ordenacao=ID_UNICO
2025-10-16 14:01:25 - WARNING - Erro 429. Tentativa 1/5. Aguardando 180s...
2025-10-16 14:04:25 - INFO - Requisitando URL: https://api.obrasgov.gestao.gov.br/obrasgov/api/projeto-investimento?uf=DF&pagina=0&tamanhoDaPagina=100&ordenacao=ID_UNICO
2025-10-16 14:04:25 - WARNING - Erro 429. Tentativa 2/5. Aguardando 360s...
2025-10-16 18:31:05 - INFO - Requisitando URL: https://api.obrasgov.gestao.gov.br/obrasgov/api/projeto-investimento?uf=DF&pagina=0&tamanhoDaPagina=100&ordenacao=ID_UNICO
2025-10-16 18:34:05 - INFO - Requisitando URL: https://api.obrasgov.gestao.gov.br/obrasgov/api/projeto-investimento?uf=DF&pagina=1&tamanhoDaPagina=100&ordenacao=ID_UNICO
2025-10-16 18:34:05 - WARNING - Erro 429. Tentativa 1/5. Aguardando 180s...
2025-10-16 18:37:28 - INFO - Requisitando URL: https://api.obrasgov.gestao.gov.br/obrasgov/a


Dados salvos com sucesso em formato JSON: ../data/raw/dados_obras_gov.json
Total de registros salvos: 700


In [ ]:
# Transformando tudo em DataFrame
df = pd.json_normalize(data)
print(df.head())
print("Total de registros: ", len(df))

       idUnico                                               nome         cep  \
0  50379.53-54  DL - 304/2024 - Contratação de instituição par...        None   
1  42724.53-27                  Escola Classe Crixá São Sebastião        None   
2  19970.53-78  Reajuste do Contrato 45/2021 - Contrução do Ce...  70.602-600   
3  24797.53-15  Implantação de Passarelas nas Estradas Parque ...        None   
4  24822.53-70  obra de construção da  Cabine de Medição, loca...        None   

                                   endereco  \
0                                      None   
1                                      None   
2  SAIS Área Especial 3, Setor Policial Sul   
3                                      None   
4                                      None   

                                           descricao  \
0  Contratação de instituição para execução de se...   
1  Construção de Escola em Tempo Integral, Escola...   
2  Reajuste do Contrato 45/2021 - Construção do C...   
3  Imp

#### Só tem 10 registros com label DF.

In [11]:
df.describe()

,idUnico,nome,cep,endereco,descricao,funcaoSocial,metaGlobal,dataInicialPrevista,dataFinalPrevista,dataInicialEfetiva,...,observacoesPertinentes,isModeladaPorBim,dataSituacao,tomadores,executores,repassadores,eixos,tipos,subTipos,fontesDeRecurso
count,700,700,352,385,700,700,700,698,698,16,...,71,483,700,700,700,700,700,700,700,700
unique,584,557,90,221,548,435,427,377,410,10,...,4,2,339,45,75,52,18,49,75,456
top,58216.53-96,CONSTRUÇÃO DE UNIDADE BÁSICA DE SAÚDE,1,,CONSTRUÇÃO DE UNIDADE BÁSICA DE SAÚDE,Segurança Pública,Escola de Educação Infantil Tipo B,2025-06-01,2027-06-01,2018-05-17,...,Informações Obras Fundo Nacional de Desenvolvi...,False,2025-07-25,[],[{'nome': 'DEPARTAMENTO NACIONAL DE INFRAESTRU...,[],"[{'id': 1, 'descricao': 'Administrativo'}]","[{'id': 5, 'descricao': 'Administrativo', 'idE...","[{'id': 59, 'descricao': 'Obras em Imóveis de ...","[{'origem': 'Federal', 'valorInvestimentoPrevi..."
freq,3,8,78,65,8,39,40,37,36,3,...,67,450,45,397,68,289,308,128,173,81


obs: Colunas so com valores nulos

- [ ] Dados nulos devem ser analisados e possivelmente preenchidos com (mean ou median)
- [ ] Dados de data devem alterar para DateTime
- [ ] Variaveis (is) precisam alterar para bool
- [ ] Diversas colunas tem que passar por one-hot ou label encoding
- [ ] Deve ser dropada algumas colunas como idUnico (nao informativa)

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700 entries, 0 to 699
Data columns (total 31 columns):
 #   Column                              Non-Null Count  Dtype 
---  ------                              --------------  ----- 
 0   idUnico                             700 non-null    object
 1   nome                                700 non-null    object
 2   cep                                 352 non-null    object
 3   endereco                            385 non-null    object
 4   descricao                           700 non-null    object
 5   funcaoSocial                        700 non-null    object
 6   metaGlobal                          700 non-null    object
 7   dataInicialPrevista                 698 non-null    object
 8   dataFinalPrevista                   698 non-null    object
 9   dataInicialEfetiva                  16 non-null     object
 10  dataFinalEfetiva                    5 non-null      object
 11  dataCadastro                        700 non-null    object

Por hoje fiz só analise do que precisa ser feito e extração dos dados da api :)

In [7]:
cols_to_convert = ['dataInicialPrevista','dataFinalPrevista','dataCadastro','dataSituacao']

for col in cols_to_convert:
    df[col] = pd.to_datetime(df[col], errors='coerce')

KeyError: 'dataInicialPrevista'